In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
movies = pd.read_csv('D:/Intership/Recommendation_system/dataset/tmdb_5000_movies.csv') 
credits = pd.read_csv('D:/Intership/Recommendation_system/dataset/tmdb_5000_credits.csv')

In [39]:
movies.head(3)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [40]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [41]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [42]:
movies = movies.merge(credits, left_on='id', right_on='movie_id')
movies = movies[['id', 'title_x', 'genres', 'keywords', 'overview', 'vote_average', 'vote_count', 'popularity']]

movies.rename(columns={'title_x': 'title'}, inplace=True)


In [43]:
movies.dropna(subset=['overview', 'genres'], inplace=True)

In [44]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4800 non-null   int64  
 1   title         4800 non-null   object 
 2   genres        4800 non-null   object 
 3   keywords      4800 non-null   object 
 4   overview      4800 non-null   object 
 5   vote_average  4800 non-null   float64
 6   vote_count    4800 non-null   int64  
 7   popularity    4800 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 337.5+ KB


In [45]:
movies.shape

(4800, 8)

In [46]:
movies.head(5)

,id,title,genres,keywords,overview,vote_average,vote_count,popularity
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.2,11800,150.437577
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.9,4500,139.082615
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.3,4466,107.376788
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.6,9106,112.312950
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.1,2124,43.926995


In [47]:
import ast

def extract_names(obj):
    try:
        data = ast.literal_eval(obj)
        return [item['name'] for item in data]
    except:
        return []

In [48]:
movies['genres'] = movies['genres'].apply(extract_names)
movies['keywords'] = movies['keywords'].apply(extract_names)


In [49]:
movies['tags'] = movies.apply(
    lambda x: ' '.join(x['genres']) + ' ' + ' '.join(x['keywords']) + ' ' + x['overview'], axis=1
)
movies = movies[['id', 'title', 'tags']]


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
vectorized_tags = vectorizer.fit_transform(movies['tags']).toarray()


In [53]:
cosine_sim = cosine_similarity(vectorized_tags)
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies['title'], columns=movies['title'])

In [51]:
def recommend(movie_title, similarity_matrix, top_n=10):

    if movie_title not in similarity_matrix.index:
        return ["Movie not found in dataset"]

    # Retrieve similarity scores and sort
    sim_scores = similarity_matrix[movie_title].sort_values(ascending=False)

    # Exclude the input movie and get top N recommendations
    recommendations = sim_scores.iloc[1:top_n+1].index.tolist()
    return recommendations

In [52]:
print(recommend('The Dark Knight', cosine_sim_df, top_n=5))

['The Dark Knight Rises', 'Batman Returns', 'Batman Begins', 'Batman Forever', 'Batman v Superman: Dawn of Justice']
